In [1]:
from model_teams import NeuralNetTeams
from model_pairs import NeuralNetPairs
from itertools import combinations
import random

import os
import torch
import numpy as np

In [2]:
file_name = "pokedex.txt"
print(file_name, '\n')

start = 0
pokedex = {}
pokedex2 = {}

file = open(file_name, 'r')
file2 = open("pokedex_dict.txt", 'w')
    
Lines = file.readlines()

for i in range(len(Lines)):
    index = Lines[start].find('\t')
    num = Lines[start][index + 2:index + 5]
    idx = Lines[start][index + 6:].find('\t')
    poke_name = Lines[start][index + 6: index + 6 + idx]
    
    file2.write(str(start))
#     file2.write(num)
    file2.write('\n')
    file2.write(poke_name)
    file2.write('\n')
    
    pokedex[poke_name] = start
    pokedex2[start] = poke_name
    
    start += 1
    
file.close()
file2.close()

pokedex

pokedex.txt 



{'Bulbasaur': 0,
 'Ivysaur': 1,
 'Venusaur': 2,
 'Charmander': 3,
 'Charmeleon': 4,
 'Charizard': 5,
 'Squirtle': 6,
 'Wartortle': 7,
 'Blastoise': 8,
 'Caterpie': 9,
 'Metapod': 10,
 'Butterfree': 11,
 'Weedle': 12,
 'Kakuna': 13,
 'Beedrill': 14,
 'Pidgey': 15,
 'Pidgeotto': 16,
 'Pidgeot': 17,
 'Rattata': 18,
 'Rattata-Alola': 19,
 'Raticate': 20,
 'Raticate-Alola': 21,
 'Spearow': 22,
 'Fearow': 23,
 'Ekans': 24,
 'Arbok': 25,
 'Pikachu': 26,
 'Raichu': 27,
 'Raichu-Alola': 28,
 'Sandshrew': 29,
 'Sandshrew-Alola': 30,
 'Sandslash': 31,
 'Sandslash-Alola': 32,
 'Nidoranâ™€': 33,
 'Nidorina': 34,
 'Nidoqueen': 35,
 'Nidoranâ™‚': 36,
 'Nidorino': 37,
 'Nidoking': 38,
 'Clefairy': 39,
 'Clefable': 40,
 'Vulpix': 41,
 'Vulpix-Alola': 42,
 'Ninetales': 43,
 'Ninetales-Alola': 44,
 'Jigglypuff': 45,
 'Wigglytuff': 46,
 'Zubat': 47,
 'Golbat': 48,
 'Oddish': 49,
 'Gloom': 50,
 'Vileplume': 51,
 'Paras': 52,
 'Parasect': 53,
 'Venonat': 54,
 'Venomoth': 55,
 'Diglett': 56,
 'Diglett-Alola'

In [3]:
%%time

dir_name = "Game_Profiles/"
team_name = "trick room"
full_path = dir_name + team_name

print(full_path, '\n\n')
data_teams = np.empty((0,979))
labels_teams = []

data_pairs = np.empty((0,979))
labels_pairs = []

da_stats = {}

start = 0
for filename in os.listdir(full_path):
    file = open(full_path + '/' + filename, 'r')
    print(filename)
    Lines = file.readlines()
    
    for i in range(len(Lines)):
        if(Lines[i] == "--------------------\n"):
            start = i + 1
            break
        else:
            continue
            
    poketeam = []        
    
    # Collect all pokemon on enemy team
    while(start < len(Lines)):
        
        if "No more" in Lines[start]:
            break
        
        # Get lead status
        if "lead" in Lines[start].strip():
            status = 2
        elif "back" in Lines[start].strip():
            status = 1
        else:
            status = 0
        
        start += 1 # Go to name
        
        #Get name
        poke_name = Lines[start].strip()
        
        if ("Regi" in poke_name): 
            print(poke_name)
            print(filename)

        start += 13 #skip to next pokemon
        
        print('{:14.14s}'.format(poke_name), '\t', pokedex[poke_name], end='\t')
        if status == 2:
            print("lead")
        elif status == 1:
            print("back")
        else:
            print("absent")
        poketeam.append((pokedex[poke_name], status))
    
    
    # Make input and output vector
    team = torch.zeros(979)
    
    lead = []
    back = []
    absent = []
    
    for (poke, stat) in poketeam:
        team[poke] = 1
        if stat == 2:
            lead.append(poke)
        elif stat == 1:
            back.append(poke)
        else:
            absent.append(poke)
    
    data_teams = np.vstack([data_teams, team])
    labels_teams.append((absent, back, lead) )
    
#     print(poketeam, '\n')

    # Make all combinations
    for combo in combinations(poketeam, 2):  
        pair = torch.zeros(979)
        pair[combo[0][0]] = 1
        pair[combo[1][0]] = 1
        print(combo, end='\t')
        if combo[0][1] == 2 and combo[1][1] == 2:
            print("This is the lead", end='')
        
        isLead = int(combo[0][1] == 2 and combo[1][1] == 2)
        
        data_pairs = np.vstack([data_pairs, pair])
        labels_pairs.append(torch.tensor(isLead))
        print()
    print()
    
    for (poke1, s1) in poketeam:
        if (poke1 == 972): 
            print("Regieleki")
        if (poke1 == 973): 
            print("Regidrago")
        poke1 = pokedex2[poke1]
        
        if poke1 not in da_stats:
            da_stats[poke1] = {}
        for (poke2, s2) in poketeam:
            poke2 = pokedex2[poke2]
            if poke2 in da_stats[poke1]:
                da_stats[poke1][poke2] += 1
            else:
                da_stats[poke1][poke2] = 1
            
            if s1 == 2 and s2 == 2:
                if "lead" not in da_stats[poke1]:
                    da_stats[poke1]["lead"] = {}
                if poke2 not in da_stats[poke1]["lead"]:
                    da_stats[poke1]["lead"][poke2] = 1
                else:
                    da_stats[poke1]["lead"][poke2] += 1
            
            if s1 == 1 and s2 == 1:
                if "back" not in da_stats[poke1]:
                    da_stats[poke1]["back"] = {}
                if poke2 not in da_stats[poke1]["back"]:
                    da_stats[poke1]["back"][poke2] = 1
                else:
                    da_stats[poke1]["back"][poke2] += 1
            
            if s1 == 0 and s2 == 0:
                if "absent" not in da_stats[poke1]:
                    da_stats[poke1]["absent"] = {}
                if poke2 not in da_stats[poke1]["absent"]:
                    da_stats[poke1]["absent"][poke2] = 1
                else:
                    da_stats[poke1]["absent"][poke2] += 1
                
    

labels_pairs = np.array(labels_pairs)

file.close()

Game_Profiles/trick room 


GameProfile000000
Hatterene      	 932	lead
Indeedee-F     	 950	absent
Liepard        	 562	lead
Dusclops       	 391	absent
Torkoal        	 356	absent
Dhelmise       	 853	absent
((932, 2), (950, 0))	
((932, 2), (562, 2))	This is the lead
((932, 2), (391, 0))	
((932, 2), (356, 0))	
((932, 2), (853, 0))	
((950, 0), (562, 2))	
((950, 0), (391, 0))	
((950, 0), (356, 0))	
((950, 0), (853, 0))	
((562, 2), (391, 0))	
((562, 2), (356, 0))	
((562, 2), (853, 0))	
((391, 0), (356, 0))	
((391, 0), (853, 0))	
((356, 0), (853, 0))	

GameProfile000001
Reuniclus      	 638	lead
Marowak-Alola  	 128	absent
Conkeldurr     	 587	back
Indeedee-F     	 950	absent
Tyranitar      	 278	absent
Primarina      	 797	lead
((638, 2), (128, 0))	
((638, 2), (587, 1))	
((638, 2), (950, 0))	
((638, 2), (278, 0))	
((638, 2), (797, 2))	This is the lead
((128, 0), (587, 1))	
((128, 0), (950, 0))	
((128, 0), (278, 0))	
((128, 0), (797, 2))	
((587, 1), (950, 0))	
((587, 1), (278, 0))	
((587

((2, 0), (134, 0))	
((794, 0), (356, 0))	
((794, 0), (962, 2))	
((794, 0), (134, 0))	
((356, 0), (962, 2))	
((356, 0), (134, 0))	
((962, 2), (134, 0))	

GameProfile000018
Cinderace      	 887	lead
Tyranitar      	 278	back
Togekiss       	 513	absent
Milotic        	 382	lead
Incineroar     	 794	back
Porygon-Z      	 519	absent
((887, 2), (278, 1))	
((887, 2), (513, 0))	
((887, 2), (382, 2))	This is the lead
((887, 2), (794, 1))	
((887, 2), (519, 0))	
((278, 1), (513, 0))	
((278, 1), (382, 2))	
((278, 1), (794, 1))	
((278, 1), (519, 0))	
((513, 0), (382, 2))	
((513, 0), (794, 1))	
((513, 0), (519, 0))	
((382, 2), (794, 1))	
((382, 2), (519, 0))	
((794, 1), (519, 0))	

GameProfile000019
Tyranitar      	 278	absent
Grimmsnarl     	 935	lead
Gyarados       	 155	back
Arcanine       	 70	absent
Duraludon      	 959	absent
Amoonguss      	 650	lead
((278, 0), (935, 2))	
((278, 0), (155, 1))	
((278, 0), (70, 0))	
((278, 0), (959, 0))	
((278, 0), (650, 2))	
((935, 2), (155, 1))	
((935, 2), (

((519, 1), (727, 0))	
((39, 0), (970, 2))	
((39, 0), (797, 2))	
((39, 0), (70, 1))	
((39, 0), (727, 0))	
((970, 2), (797, 2))	This is the lead
((970, 2), (70, 1))	
((970, 2), (727, 0))	
((797, 2), (70, 1))	
((797, 2), (727, 0))	
((70, 1), (727, 0))	

GameProfile000036
Excadrill      	 583	lead
Rotom-Wash     	 526	back
Incineroar     	 794	lead
Dragapult      	 962	absent
Togekiss       	 513	absent
Amoonguss      	 650	back
((583, 2), (526, 1))	
((583, 2), (794, 2))	This is the lead
((583, 2), (962, 0))	
((583, 2), (513, 0))	
((583, 2), (650, 1))	
((526, 1), (794, 2))	
((526, 1), (962, 0))	
((526, 1), (513, 0))	
((526, 1), (650, 1))	
((794, 2), (962, 0))	
((794, 2), (513, 0))	
((794, 2), (650, 1))	
((962, 0), (513, 0))	
((962, 0), (650, 1))	
((513, 0), (650, 1))	

GameProfile000037
Primarina      	 797	absent
Cobalion       	 700	lead
Arcanine       	 70	absent
Grimmsnarl     	 935	absent
Hydreigon      	 697	absent
Persian-Alola  	 64	lead
((797, 0), (700, 2))	
((797, 0), (70, 0))	
(

((214, 1), (525, 0))	
((214, 1), (935, 0))	
((260, 2), (659, 1))	
((260, 2), (970, 2))	This is the lead
((260, 2), (525, 0))	
((260, 2), (935, 0))	
((659, 1), (970, 2))	
((659, 1), (525, 0))	
((659, 1), (935, 0))	
((970, 2), (525, 0))	
((970, 2), (935, 0))	
((525, 0), (935, 0))	

GameProfile000053
Dracozolt      	 955	absent
Corviknight    	 895	back
Arcanine       	 70	lead
Gastrodon      	 468	absent
Grimmsnarl     	 935	lead
Amoonguss      	 650	absent
((955, 0), (895, 1))	
((955, 0), (70, 2))	
((955, 0), (468, 0))	
((955, 0), (935, 2))	
((955, 0), (650, 0))	
((895, 1), (70, 2))	
((895, 1), (468, 0))	
((895, 1), (935, 2))	
((895, 1), (650, 0))	
((70, 2), (468, 0))	
((70, 2), (935, 2))	This is the lead
((70, 2), (650, 0))	
((468, 0), (935, 2))	
((468, 0), (650, 0))	
((935, 2), (650, 0))	

GameProfile000054
Dracozolt      	 955	absent
Corviknight    	 895	back
Arcanine       	 70	lead
Gastrodon      	 468	absent
Grimmsnarl     	 935	lead
Amoonguss      	 650	back
((955, 0), (895, 1))	

((932, 2), (848, 1))	
((932, 2), (241, 1))	
((950, 2), (650, 0))	
((950, 2), (757, 0))	
((950, 2), (848, 1))	
((950, 2), (241, 1))	
((650, 0), (757, 0))	
((650, 0), (848, 1))	
((650, 0), (241, 1))	
((757, 0), (848, 1))	
((757, 0), (241, 1))	
((848, 1), (241, 1))	

GameProfile000072
Clefairy       	 39	lead
Porygon-Z      	 519	absent
Azumarill      	 212	lead
Roserade       	 445	absent
Talonflame     	 727	absent
Amoonguss      	 650	absent
((39, 2), (519, 0))	
((39, 2), (212, 2))	This is the lead
((39, 2), (445, 0))	
((39, 2), (727, 0))	
((39, 2), (650, 0))	
((519, 0), (212, 2))	
((519, 0), (445, 0))	
((519, 0), (727, 0))	
((519, 0), (650, 0))	
((212, 2), (445, 0))	
((212, 2), (727, 0))	
((212, 2), (650, 0))	
((445, 0), (727, 0))	
((445, 0), (650, 0))	
((727, 0), (650, 0))	

GameProfile000073
Clefairy       	 39	back
Porygon-Z      	 519	lead
Azumarill      	 212	back
Roserade       	 445	absent
Talonflame     	 727	lead
Amoonguss      	 650	absent
((39, 1), (519, 2))	
((39, 1), (212

...

((28, 0), (513, 1))	
((887, 2), (970, 2))	This is the lead
((887, 2), (945, 0))	
((887, 2), (950, 1))	
((887, 2), (513, 1))	
((970, 2), (945, 0))	
((970, 2), (950, 1))	
((970, 2), (513, 1))	
((945, 0), (950, 1))	
((945, 0), (513, 1))	
((950, 1), (513, 1))	

GameProfile000370
Whimsicott     	 600	absent
Cinderace      	 887	lead
Duraludon      	 959	lead
Tyranitar      	 278	back
Incineroar     	 794	absent
Rillaboom      	 884	back
((600, 0), (887, 2))	
((600, 0), (959, 2))	
((600, 0), (278, 1))	
((600, 0), (794, 0))	
((600, 0), (884, 1))	
((887, 2), (959, 2))	This is the lead
((887, 2), (278, 1))	
((887, 2), (794, 0))	
((887, 2), (884, 1))	
((959, 2), (278, 1))	
((959, 2), (794, 0))	
((959, 2), (884, 1))	
((278, 1), (794, 0))	
((278, 1), (884, 1))	
((794, 0), (884, 1))	

GameProfile000371
Whimsicott     	 600	absent
Cinderace      	 887	lead
Duraludon      	 959	lead
Tyranitar      	 278	back
Incineroar     	 794	absent
Rillaboom      	 884	back
((600, 0), (887, 2))	
((600, 0), (959, 

((835, 2), (957, 2))	This is the lead
((835, 2), (650, 1))	
((129, 0), (168, 1))	
((129, 0), (957, 2))	
((129, 0), (650, 1))	
((168, 1), (957, 2))	
((168, 1), (650, 1))	
((957, 2), (650, 1))	

GameProfile000386
Milotic        	 382	lead
Arcanine       	 70	back
Raichu         	 27	absent
Kommo-o        	 856	lead
Hatterene      	 932	back
Cobalion       	 700	absent
((382, 2), (70, 1))	
((382, 2), (27, 0))	
((382, 2), (856, 2))	This is the lead
((382, 2), (932, 1))	
((382, 2), (700, 0))	
((70, 1), (27, 0))	
((70, 1), (856, 2))	
((70, 1), (932, 1))	
((70, 1), (700, 0))	
((27, 0), (856, 2))	
((27, 0), (932, 1))	
((27, 0), (700, 0))	
((856, 2), (932, 1))	
((856, 2), (700, 0))	
((932, 1), (700, 0))	

GameProfile000387
Clefairy       	 39	absent
Scizor         	 241	absent
Chandelure     	 670	lead
Corviknight    	 895	lead
Tyranitar      	 278	absent
Gastrodon-East 	 466	absent
((39, 0), (241, 0))	
((39, 0), (670, 2))	
((39, 0), (895, 2))	
((39, 0), (278, 0))	
((39, 0), (466, 0))	
((241, 0

((884, 1), (382, 0))	
((884, 1), (129, 1))	
((884, 1), (727, 2))	
((884, 1), (890, 0))	
((382, 0), (129, 1))	
((382, 0), (727, 2))	
((382, 0), (890, 0))	
((129, 1), (727, 2))	
((129, 1), (890, 0))	
((727, 2), (890, 0))	

GameProfile000402
Indeedee-F     	 950	absent
Hatterene      	 932	back
Conkeldurr     	 587	lead
Incineroar     	 794	absent
Excadrill      	 583	back
Porygon2       	 263	lead
((950, 0), (932, 1))	
((950, 0), (587, 2))	
((950, 0), (794, 0))	
((950, 0), (583, 1))	
((950, 0), (263, 2))	
((932, 1), (587, 2))	
((932, 1), (794, 0))	
((932, 1), (583, 1))	
((932, 1), (263, 2))	
((587, 2), (794, 0))	
((587, 2), (583, 1))	
((587, 2), (263, 2))	This is the lead
((794, 0), (583, 1))	
((794, 0), (263, 2))	
((583, 1), (263, 2))	

GameProfile000403
Whimsicott     	 600	lead
Amoonguss      	 650	absent
Jellicent      	 654	back
Cobalion       	 700	lead
Conkeldurr     	 587	absent
Hydreigon      	 697	back
((600, 2), (650, 0))	
((600, 2), (654, 1))	
((600, 2), (700, 2))	This is the

((356, 0), (950, 0))	
((356, 0), (932, 0))	
((2, 0), (877, 0))	
((2, 0), (950, 0))	
((2, 0), (932, 0))	
((877, 0), (950, 0))	
((877, 0), (932, 0))	
((950, 0), (932, 0))	

GameProfile000418
Pincurchin     	 945	absent
Raichu-Alola   	 28	back
Porygon-Z      	 519	lead
Clefairy       	 39	back
Incineroar     	 794	absent
Chandelure     	 670	lead
((945, 0), (28, 1))	
((945, 0), (519, 2))	
((945, 0), (39, 1))	
((945, 0), (794, 0))	
((945, 0), (670, 2))	
((28, 1), (519, 2))	
((28, 1), (39, 1))	
((28, 1), (794, 0))	
((28, 1), (670, 2))	
((519, 2), (39, 1))	
((519, 2), (794, 0))	
((519, 2), (670, 2))	This is the lead
((39, 1), (794, 0))	
((39, 1), (670, 2))	
((794, 0), (670, 2))	

GameProfile000419
Cramorant      	 917	lead
Wigglytuff     	 46	absent
Flapple        	 913	absent
Togetic        	 204	lead
Silvally-*     	 845	back
Sandaconda     	 916	absent
((917, 2), (46, 0))	
((917, 2), (913, 0))	
((917, 2), (204, 2))	This is the lead
((917, 2), (845, 1))	
((917, 2), (916, 0))	
((46, 0), (9

((156, 2), (951, 2))	This is the lead
((156, 2), (600, 0))	
((767, 1), (884, 1))	
((767, 1), (278, 0))	
((767, 1), (951, 2))	
((767, 1), (600, 0))	
((884, 1), (278, 0))	
((884, 1), (951, 2))	
((884, 1), (600, 0))	
((278, 0), (951, 2))	
((278, 0), (600, 0))	
((951, 2), (600, 0))	

GameProfile000434
Indeedee-F     	 950	back
Rillaboom      	 884	absent
Riolu          	 492	absent
Hatterene      	 932	back
Incineroar     	 794	lead
Cinderace      	 887	lead
((950, 1), (884, 0))	
((950, 1), (492, 0))	
((950, 1), (932, 1))	
((950, 1), (794, 2))	
((950, 1), (887, 2))	
((884, 0), (492, 0))	
((884, 0), (932, 1))	
((884, 0), (794, 2))	
((884, 0), (887, 2))	
((492, 0), (932, 1))	
((492, 0), (794, 2))	
((492, 0), (887, 2))	
((932, 1), (794, 2))	
((932, 1), (887, 2))	
((794, 2), (887, 2))	This is the lead

GameProfile000435
Drapion        	 497	absent
Kingler        	 120	absent
Togekiss       	 513	lead
Chandelure     	 670	back
Excadrill      	 583	lead
Rillaboom      	 884	back
((497, 0), (120,

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if (torch.cuda.is_available()):
  print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()
device

GeForce GTX 1660 SUPER


device(type='cuda')

In [5]:
test_size = 0

copy_data = np.copy(data_teams)
copy_labels = labels_teams.copy()

index = np.random.choice(len(copy_data), size=test_size, replace=False).tolist()
index.sort()
index = index[::-1]

test_data = copy_data[index]
test_labels = [copy_labels[x] for x in index]

train_data = np.delete(copy_data, index, axis=0)
train_labels = labels_teams.copy()
for x in index:
    train_labels.pop(x)

print(data_teams.shape)
print(test_data.shape)
print(train_data.shape)

test_data = torch.from_numpy(test_data).to(device)
# test_labels = torch.from_numpy(test_labels).to(device).view(-1, 1)

train_data = torch.from_numpy(train_data).to(device)
# train_labels = torch.from_numpy(train_labels).to(device).view(-1, 1)

(440, 979)
(0, 979)
(440, 979)


In [6]:
def fitTeams(net=None, num_epoch=10, batch_size=10):
    """ Fit a neural net.  Use the full batch size.
    
    @param batch_size: The size of each batch to train on.
    @param num_epoch: The number of epochs to train on
    @param net: A NeuralNet object
    
    @return losses: losses for each batch
    @return net: A NeuralNet object
    """
    
    # Defining the neural network

    if (net == None):
        net = NeuralNetTeams(0.01)
    net.to(device)
    
    # Training

    losses = []
    for epoch in range(num_epoch):
        # Training
        i = 0
        print("epoch:", epoch)
        while (i + 1) * batch_size < len(train_data) + 1:
        
            batch = train_data[i * batch_size: (i + 1) * batch_size]
            target = train_labels[i * batch_size: (i + 1) * batch_size]
        
            batch = batch.float().to(device)
                    
            loss = net.step(batch, target)
            losses.append(loss)
            print('Loss', i, ':', "{:.5f}".format(loss), end='\r')

            i += 1
        print()
        
    return losses, net

In [7]:
%%time
losses, mynet = fitTeams()

epoch: 0
Loss 43 : 38.69436
epoch: 1
Loss 43 : 37.96214
epoch: 2
Loss 43 : 37.51947
epoch: 3
Loss 43 : 37.24799
epoch: 4
Loss 43 : 37.0164314 : 38.03139
epoch: 5
Loss 43 : 36.77077
epoch: 6
Loss 43 : 36.43262
epoch: 7
Loss 43 : 36.0207436.07825
epoch: 8
Loss 43 : 35.5750431 : 32.31126
epoch: 9
Loss 43 : 35.08405
Wall time: 2.57 s


In [8]:
x = random.randint(0, len(data_teams))
print(x)
output = mynet(torch.from_numpy(data_teams[x]).float().to(device))

print(output[0, labels_teams[x][0]])
print(output[0, labels_teams[x][1]])
print(output[0, labels_teams[x][2]])

print(labels_teams[x])
for i in labels_teams[x]:
    for j in i:
        print('{:8.12s}'.format(pokedex2[j]), '\t', j)

193
tensor([[0.5576, 0.1586, 0.2838],
        [0.6306, 0.2024, 0.1670],
        [0.2835, 0.1763, 0.5402]], device='cuda:0', grad_fn=<IndexBackward>)
tensor([[0.2936, 0.1086, 0.5978]], device='cuda:0', grad_fn=<IndexBackward>)
tensor([[0.3269, 0.2734, 0.3997],
        [0.3944, 0.4179, 0.1877]], device='cuda:0', grad_fn=<IndexBackward>)
([794, 583, 970], [951], [797, 513])
Incineroar 	 794
Excadrill 	 583
Urshifu-* 	 970
Indeedee 	 951
Primarina 	 797
Togekiss 	 513


In [9]:
team = torch.zeros(979)
team[957] = 1
team[583] = 1
team[857] = 1
team[5] = 1
team[391] = 1
team[659] = 1

output = mynet(team.float().to(device))

print(output[0, 957])
print(output[0, 5])
print(output[0, 583])
print(output[0, 857])
print(output[0, 214])
print(output[0, 600])

print('{:4.12s}'.format(pokedex2[957]))
print('{:4.12s}'.format(pokedex2[5]))
print('{:4.12s}'.format(pokedex2[583]))
print('{:4.12s}'.format(pokedex2[857]))
print('{:4.12s}'.format(pokedex2[391]))
print('{:4.12s}'.format(pokedex2[659]))

tensor([0.2779, 0.4162, 0.3059], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.3364, 0.2122, 0.4514], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.5135, 0.2767, 0.2098], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.3408, 0.3618, 0.2974], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.5678, 0.2688, 0.1633], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.6355, 0.1258, 0.2387], device='cuda:0', grad_fn=<SelectBackward>)
Dracovish
Charizard
Excadrill
Tapu Koko
Dusclops
Ferrothorn


In [10]:
test_size = 0

copy_data = np.copy(data_pairs)
copy_labels = np.copy(labels_pairs)

index = np.random.choice(len(copy_data), size=test_size, replace=False).tolist()

test_data = copy_data[index]
test_labels = copy_labels[index]

train_data = np.delete(copy_data, index, axis=0)
train_labels = np.delete(copy_labels, index, axis=0)

print(data_pairs.shape)
print(test_data.shape)
print(train_data.shape)
print(train_labels.shape)

test_data = torch.from_numpy(test_data).to(device)
test_labels = torch.from_numpy(test_labels).to(device).view(-1, 1)

train_data = torch.from_numpy(train_data).to(device)
train_labels = torch.from_numpy(train_labels).to(device).view(-1, 1)

(6582, 979)
(0, 979)
(6582, 979)
(6582,)


In [17]:
def fitPairs(net=None, num_epoch=1000, batch_size=10):
    """ Fit a neural net.  Use the full batch size.
    
    @param batch_size: The size of each batch to train on.
    @param num_epoch: The number of epochs to train on
    @param net: A NeuralNet object
    
    @return losses: losses for each batch
    @return net: A NeuralNet object
    """
    
    # Defining the neural network

    if (net == None):
        net = NeuralNetPairs(0.0005)
    net.to(device)
    
    # Training

    losses = []
    for epoch in range(num_epoch):
        # Training
        i = 0
        print("epoch:", epoch)
        while (i + 1) * batch_size < len(train_data) + 1:
        
            batch = train_data[i * batch_size: (i + 1) * batch_size]
            target = train_labels[i * batch_size: (i + 1) * batch_size]
        
            batch = batch.float().to(device)
            target = target.float().to(device)
                    
            loss = net.step(batch, target)
            losses.append(loss)
            if i % 25 == 0:
                print('Loss', i, ':', "{:.5f}".format(loss), end='\r')

            i += 1
        print()
        
    return losses, net

In [87]:
%%time
losses, mynet2 = fitPairs(mynet2)

epoch: 0
Loss 650 : 0.06789
epoch: 1
Loss 650 : 0.06791300 : 0.98737
epoch: 2
Loss 650 : 0.06785
epoch: 3
Loss 650 : 0.06769
epoch: 4
Loss 650 : 0.06808.57879575 : 0.05229
epoch: 5
Loss 650 : 0.06778
epoch: 6
Loss 650 : 0.06788
epoch: 7
Loss 650 : 0.06782.46958
epoch: 8
Loss 650 : 0.06803600 : 0.83179
epoch: 9
Loss 650 : 0.06811
epoch: 10
Loss 650 : 0.06808
epoch: 11
Loss 650 : 0.06833
epoch: 12
Loss 650 : 0.06823
epoch: 13
Loss 650 : 0.06828
epoch: 14
Loss 650 : 0.06830 : 0.39436450 : 0.19208
epoch: 15
Loss 650 : 0.06830
epoch: 16
Loss 650 : 0.06795
epoch: 17
Loss 650 : 0.06793
epoch: 18
Loss 650 : 0.06799
epoch: 19
Loss 650 : 0.06807
epoch: 20
Loss 650 : 0.068105 : 0.57955
epoch: 21
Loss 650 : 0.06813375 : 0.96126
epoch: 22
Loss 650 : 0.06816
epoch: 23
Loss 650 : 0.068180.05185
epoch: 24
Loss 650 : 0.06802
epoch: 25
Loss 650 : 0.06788
epoch: 26
Loss 650 : 0.06818
epoch: 27
Loss 650 : 0.06801
epoch: 28
Loss 650 : 0.06790
epoch: 29
Loss 650 : 0.06789: 0.72448
epoch: 30
Loss 650 : 0.068

Loss 650 : 0.06742150 : 0.90701600 : 0.83507
epoch: 228
Loss 650 : 0.06764
epoch: 229
Loss 650 : 0.06729
epoch: 230
Loss 650 : 0.06753
epoch: 231
Loss 650 : 0.067620.39201
epoch: 232
Loss 650 : 0.06749
epoch: 233
Loss 650 : 0.06738125 : 0.12841625 : 0.03991
epoch: 234
Loss 650 : 0.06725
epoch: 235
Loss 650 : 0.06736550 : 0.04764
epoch: 236
Loss 650 : 0.06703600 : 0.83461
epoch: 237
Loss 650 : 0.06736150 : 0.90695375 : 0.96543
epoch: 238
Loss 650 : 0.06728 : 0.72185 0.04514
epoch: 239
Loss 650 : 0.06700225 : 0.39226450 : 0.18614
epoch: 240
Loss 650 : 0.06689 : 0.47123 0.02758
epoch: 241
Loss 650 : 0.066830.05193
epoch: 242
Loss 650 : 0.067080.90622
epoch: 243
Loss 650 : 0.06687225 : 0.391820.18589
epoch: 244
Loss 650 : 0.06701 : 0.47056250 : 0.13097
epoch: 245
Loss 650 : 0.066825 : 0.57907: 0.04740
epoch: 246
Loss 650 : 0.066725 : 0.57867
epoch: 247
Loss 650 : 0.06699 0.96580
epoch: 248
Loss 650 : 0.06672425 : 0.04729
epoch: 249
Loss 650 : 0.06699
epoch: 250
Loss 650 : 0.06678 0.02743
e

Loss 650 : 0.066400 : 0.69610275 : 0.77256
epoch: 443
Loss 650 : 0.06624350 : 0.74862 : 0.05203
epoch: 444
Loss 650 : 0.06638150 : 0.89783
epoch: 445
Loss 650 : 0.06647
epoch: 446
Loss 650 : 0.06640
epoch: 447
Loss 650 : 0.06642
epoch: 448
Loss 650 : 0.06630
epoch: 449
Loss 650 : 0.066590 : 0.69633 : 0.97731
epoch: 450
Loss 650 : 0.06658600 : 0.83625
epoch: 451
Loss 650 : 0.06644
epoch: 452
Loss 650 : 0.06659
epoch: 453
Loss 650 : 0.06639600 : 0.83605
epoch: 454
Loss 650 : 0.06641
epoch: 455
Loss 650 : 0.06644
epoch: 456
Loss 650 : 0.06672
epoch: 457
Loss 650 : 0.06640
epoch: 458
Loss 650 : 0.06637
epoch: 459
Loss 650 : 0.066500.69695: 0.77301500 : 0.02647
epoch: 460
Loss 650 : 0.06624
epoch: 461
Loss 650 : 0.06630
epoch: 462
Loss 650 : 0.06641
epoch: 463
Loss 650 : 0.06632
epoch: 464
Loss 650 : 0.06632
epoch: 465
Loss 650 : 0.06598
epoch: 466
Loss 650 : 0.03987: 0.06617
epoch: 467
Loss 650 : 0.06612225 : 0.38789
epoch: 468
Loss 650 : 0.06646
epoch: 469
Loss 650 : 0.06586
epoch: 470
Lo

Loss 650 : 0.06639175 : 0.71413
epoch: 657
Loss 650 : 0.06639475 : 0.39997
epoch: 658
Loss 650 : 0.066360 : 0.69407275 : 0.77687
epoch: 659
Loss 650 : 0.066030.74825
epoch: 660
Loss 650 : 0.06609450 : 0.17388
epoch: 661
Loss 650 : 0.06637 : 0.46657 0.12955475 : 0.39997
epoch: 662
Loss 650 : 0.06631: 0.69351275 : 0.77670
epoch: 663
Loss 650 : 0.0660600 : 0.28112
epoch: 664
Loss 650 : 0.06587
epoch: 665
Loss 650 : 0.06588475 : 0.39986
epoch: 666
Loss 650 : 0.06595
epoch: 667
Loss 650 : 0.06569 : 0.03946
epoch: 668
Loss 650 : 0.06594
epoch: 669
Loss 650 : 0.06590
epoch: 670
Loss 650 : 0.06586
epoch: 671
Loss 650 : 0.06578
epoch: 672
Loss 650 : 0.065870 : 0.69392
epoch: 673
Loss 650 : 0.06584
epoch: 674
Loss 650 : 0.06586425 : 0.04716
epoch: 675
Loss 650 : 0.06574
epoch: 676
Loss 650 : 0.06551
epoch: 677
Loss 650 : 0.06560
epoch: 678
Loss 650 : 0.06600275 : 0.77653
epoch: 679
Loss 650 : 0.0657300 : 0.28006 : 0.83980
epoch: 680
Loss 650 : 0.06574 0.71565400 : 0.04347
epoch: 681
Loss 650 : 0

Loss 650 : 0.06462
epoch: 872
Loss 650 : 0.06485 : 0.46588 0.12910: 0.40059
epoch: 873
Loss 650 : 0.06476550 : 0.04780
epoch: 874
Loss 650 : 0.06484
epoch: 875
Loss 650 : 0.06484: 0.02455
epoch: 876
Loss 650 : 0.06493
epoch: 877
Loss 650 : 0.06488125 : 0.12960
epoch: 878
Loss 650 : 0.06471175 : 0.71950
epoch: 879
Loss 650 : 0.06480475 : 0.40090
epoch: 880
Loss 650 : 0.06482
epoch: 881
Loss 650 : 0.064730.91340
epoch: 882
Loss 650 : 0.06477400 : 0.04230
epoch: 883
Loss 650 : 0.06479
epoch: 884
Loss 650 : 0.06503
epoch: 885
Loss 650 : 0.06510
epoch: 886
Loss 650 : 0.065140.87757375 : 0.97508
epoch: 887
Loss 650 : 0.06509 : 0.71955
epoch: 888
Loss 650 : 0.06508450 : 0.16786
epoch: 889
Loss 650 : 0.06505.46484250 : 0.12878450 : 0.16776
epoch: 890
Loss 650 : 0.065230.16761
epoch: 891
Loss 650 : 0.06545.46446
epoch: 892
Loss 650 : 0.06517
epoch: 893
Loss 650 : 0.06538425 : 0.04730
epoch: 894
Loss 650 : 0.06558
epoch: 895
Loss 650 : 0.06529
epoch: 896
Loss 650 : 0.06496625 : 0.03947
epoch: 89

In [95]:
x = random.randint(0, len(data_pairs) - 15)
print(x, '\n')

output = mynet2(torch.from_numpy(data_pairs[x:x+15]).float().to(device))
answers = labels_pairs[x:x+15] 

print("Predicted\tActual")
for i in range(len(answers)):
    print('{:.5f}\t\t{:d}'.format(output[i].item(), answers[i]))

1007 

Predicted	Actual
0.21886		0
0.08889		0
0.07793		0
0.05050		0
0.02123		0
0.10304		0
0.11830		0
0.05637		0
0.05357		0
0.17975		0
0.05407		0
0.72564		1
0.14438		0
0.42556		1
0.02290		0
